In [2]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 데이터 로드 (이전에 사용한 CSV 파일과 같은 데이터 사용)
data = pd.read_csv("mon.csv")

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 'Label'이 레이블 컬럼 이름
y = data['Label']

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 랜덤 포레스트 모델 정의 및 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 특징 중요도 계산 및 퍼센트 변환
feature_importances = rf_model.feature_importances_
features = X.columns
feature_importances_percent = (feature_importances / feature_importances.sum()) * 100

# 특징 중요도를 데이터프레임으로 정리
importance_df = pd.DataFrame({'Feature': features, 'Importance (%)': feature_importances_percent})
importance_df = importance_df.sort_values(by='Importance (%)', ascending=False)

# 중요도 데이터프레임 출력
print("Feature Importance (%):")
print(importance_df)


Feature Importance (%):
               Feature  Importance (%)
6        Outgoing Mean        9.142472
5         Outgoing Std        8.531271
0        Total Packets        8.030407
1     Incoming Packets        7.752285
14     Alternative Sum        7.470626
2     Outgoing Packets        7.438264
11   Inter-arrival Std        7.224337
13   Concentration Std        6.604982
4       Outgoing Ratio        6.317056
3       Incoming Ratio        6.276208
12  Concentration Mean        6.200562
7   Packets per Second        5.561201
10  Inter-arrival Mean        5.524881
8    First 30 Incoming        3.996963
9    First 30 Outgoing        3.928485


In [5]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# 데이터 로드
data = pd.read_csv("mon.csv")

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 'Label'이 레이블 컬럼 이름
y = data['Label']

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 랜덤 포레스트 모델 정의 및 학습 (특징 중요도 계산용)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 특징 중요도 계산 및 상위 10개 선택
feature_importances = rf_model.feature_importances_
features = X.columns
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 상위 10개 중요한 특징 선택
top_features = importance_df['Feature'][:10].tolist()
X_train_top = X_train[top_features]
X_val_top = X_val[top_features]
X_test_top = X_test[top_features]

# 데이터 정규화
scaler = StandardScaler()
X_train_top = scaler.fit_transform(X_train_top)
X_val_top = scaler.transform(X_val_top)
X_test_top = scaler.transform(X_test_top)

# Neural Network 모델 정의
model = Sequential([
    Dense(256, input_shape=(X_train_top.shape[1],), activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(95, activation='softmax')  # 95개의 클래스로 분류하기 위한 출력층
])

# 모델 컴파일 (학습률 조정)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
history = model.fit(X_train_top, y_train, epochs=100, batch_size=32, validation_data=(X_val_top, y_val), callbacks=[early_stopping])

# 모델 평가
test_loss, test_accuracy = model.evaluate(X_test_top, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

# 예측 및 F1 Score 계산
y_pred = np.argmax(model.predict(X_test_top), axis=1)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.2f}")

# 자세한 분류 보고서 출력
print(classification_report(y_test, y_pred))


Epoch 1/100
416/416 [==============================] - 3s 4ms/step - loss: 4.6365 - accuracy: 0.0224 - val_loss: 4.3139 - val_accuracy: 0.0505
Epoch 2/100
416/416 [==============================] - 1s 4ms/step - loss: 4.2623 - accuracy: 0.0581 - val_loss: 3.9626 - val_accuracy: 0.1305
Epoch 3/100
416/416 [==============================] - 1s 3ms/step - loss: 4.0020 - accuracy: 0.0950 - val_loss: 3.7357 - val_accuracy: 0.1793
Epoch 4/100
416/416 [==============================] - 1s 3ms/step - loss: 3.8115 - accuracy: 0.1227 - val_loss: 3.5550 - val_accuracy: 0.2102
Epoch 5/100
416/416 [==============================] - 1s 3ms/step - loss: 3.6632 - accuracy: 0.1432 - val_loss: 3.4198 - val_accuracy: 0.2221
Epoch 6/100
416/416 [==============================] - 1s 3ms/step - loss: 3.5441 - accuracy: 0.1565 - val_loss: 3.3064 - val_accuracy: 0.2288
Epoch 7/100
416/416 [==============================] - 1s 3ms/step - loss: 3.4362 - accuracy: 0.1746 - val_loss: 3.2121 - val_accuracy: 0.2544